In [ ]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

#### Make an exact solution and r.h.s.

In [ ]:
def uexact(x, y):
    u = torch.sin(2*torch.pi*x)*torch.sin(2*torch.pi*y)
    return u

def f(x, y):
    return 8 * torch.pi**2 * uexact(x, y)

In [ ]:
# try it out!
x, y = torch.meshgrid(torch.linspace(0, 1, 100),
                      torch.linspace(0, 1, 100), indexing='ij')
plt.pcolormesh(x, y, uexact(x,  y))

#### neural network

In [ ]:
class ffnn(nn.Module):
    def __init__(self, nn_width, num_hidden):
        super().__init__()
        
        self.layer_first = nn.Linear(2, nn_width)
        
        layers = []
        for _ in range(num_hidden):
            layers.append(nn.Linear(nn_width, nn_width))
        self.layer_hidden = nn.ModuleList(layers)
        
        self.layer_last = nn.Linear(nn_width, 1)
        
    def forward(self, x, y):
        xt = torch.cat([x, y], dim=1)
        activation = nn.Tanh()
        u = activation(self.layer_first(xt))
        for hidden in self.layer_hidden:
            u = activation(hidden(u))
        u = self.layer_last(u)
        return u

#### Create the model and set the optimizer

In [ ]:
torch.manual_seed(23447)
model = ffnn(60, 4)

lr = 0.001
opt = torch.optim.Adam(model.parameters(), lr)
n_epochs = 5000
loss_history = []

#### Defining the loss

In [ ]:
def loss_int(model, x, y):
    xt = torch.tensor(x, requires_grad=True).reshape(-1, 1)
    yt = torch.tensor(y, requires_grad=True).reshape(-1, 1)
    # TODO

def loss_bdy(model, x, y):
    # TODO

### Train the model

In [ ]:
for i in range(n_epochs):
    loss = loss_int(model, x, y) + loss_bdy(model, x, y)
    opt.zero_grad()
    loss_history.append(loss.item())
    
    loss.backward()
    opt.step()

    if i % 100 == 0:
        print(f'epoch {i}, loss = {loss}')

#### Plot the loss

#### Plot the solution and error